In [225]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from tqdm import tqdm


In [265]:
%matplotlib qt

def surface_friction_dynamics(patch_locs=None, patch_sizes=None, friction_values=None, is_growings=None, road_length=None):
    
    min_patch_size = 0 # m
    num_patches = 10
    max_patch_size = road_length/10 # m
    if patch_locs is None:
        # initialize random patches        
        patch_locs = np.random.uniform(0, road_length, num_patches)
        is_growings = np.random.choice([True, False], num_patches)
        patch_sizes = np.random.uniform(min_patch_size, max_patch_size, num_patches)
        friction_values = np.random.uniform(0.2, 0.6, num_patches)
        return patch_locs, patch_sizes, friction_values, is_growings
    for i in range(len(patch_locs)):
        if is_growings[i] and patch_sizes[i] >= max_patch_size:
            # make the patch shrink with probability 0.1
            if np.random.rand() < 0.1:
                is_growings[i] = False
        elif is_growings[i] and patch_sizes[i] < max_patch_size:
            patch_sizes[i] += .1
        elif not is_growings[i]:
            patch_sizes[i] -= .1
        if patch_sizes[i] <= min_patch_size and not is_growings[i]:
            patch_locs[i] = np.random.uniform(0, road_length)
            patch_sizes[i] = 0
            friction_values[i] = np.random.uniform(0.2, 0.8)
            is_growings[i] = True
    return patch_locs, patch_sizes, friction_values, is_growings

def get_friction(x, patch_locs, patch_sizes, friction_values):
    friction = np.full_like(x, 0.8)  # Initialize with default friction value of 0.1
    for i in range(len(patch_locs)):
        patch_start = patch_locs[i]
        patch_end = patch_locs[i] + patch_sizes[i]
        # Apply friction values only within the patch bounds
        mask = (x >= patch_start) & (x < patch_end)
        friction[mask] = friction_values[i]
    return friction

def intelligent_driver_model(x, v, x_, v_, v_desired, s_0, vehicle_length):
    delta = 4
    a = 3 # m/s^2 (comfortable acceleration)
    b = 8 # m/s^2 (comfortable deceleration)
    T = 1.5 # s

    s = x_ - x - vehicle_length
    delta_v = v - v_
    s_desired = s_0 + v*T + (v*delta_v) / (2 * np.sqrt(a * b))
    v_dot = a * (1 - (v/v_desired)**delta - (s_desired/s)**2)

    # make sure vechiles are not going backwards (negative velocity)
    v_dot[v <= 0] = np.maximum(v_dot[v <= 0], 0)

    is_crashed = s < 0.01


    if any(is_crashed):
        print(is_crashed)
        print(x)
        print(s)
        


    return v_dot, is_crashed

num_vehicles = 30
ring_radius = 1e2 # m
max_speed = 36 # m/s
confortable_acceleration = 3 # m/s^2
confortable_deceleration = 5 # m/s^2
desired_min_space = 1 # m
vehicle_length = 5 # m
dt = 0.1 # s
max_timesteps = 100_000 # s
is_crashed = np.zeros(num_vehicles, dtype=bool)
online_plot = True

s = np.array([0])

tries = 0
while any(s < 0.01):
    tries += 1
    theta = np.random.uniform(0, 2*np.pi, num_vehicles)
    theta = np.sort(theta)
    # v = np.random.uniform(0, max_speed, num_vehicles)
    v = np.zeros(num_vehicles)
    x = ring_radius * theta
    x_ = np.roll(x, -1)
    x_[-1] = x[0] + ring_radius * 2 * np.pi
    s = x_ - x - vehicle_length
    if tries > 1000:
        raise Exception("Could not find a valid initial condition after 1000 tries. Try increasing the ring radius or decreasing the number of vehicles.")
    # print(max(s))

print('Initial conditions set')
print('MinDistances: ',np.min(s))
# print('Positions: ',x)


num_crashed = 0
data = np.zeros((max_timesteps, num_vehicles, 2))
t = 0
i = 0
patch_locs, patch_sizes, friction_values, is_growings = surface_friction_dynamics(road_length=ring_radius * 2 * np.pi)

if online_plot:
    plt.close('all')
    fig, ax = plt.subplots(1, 1)
for i in tqdm(range(max_timesteps)):
    x_ = np.roll(x, -1)
    x_[-1] = x[0] + ring_radius * 2 * np.pi
    v_ = np.roll(v, -1)


    """Stop Sign Simulator"""
    # randomly select a vehicle to change its x_ value for a one-minute stop (assuming dt=0.1s)
    if i % 60 == 0:
        idx = np.random.randint(0, num_vehicles)
        fixed_pos = x_[idx]
        
    x_[idx] = fixed_pos
    v_[idx] = 0

    """ Crash Simulator"""
    # if vehicle is_crashed, set its velocity to 0 for the rest of the simulation
    if any(is_crashed):
        # print(is_crashed)
        idx = np.where(is_crashed)[0]
        v[idx] = 0



    """ Intelligent Driver Model"""
    driver_accelerations, is_crashed_new = intelligent_driver_model(x, v, x_, v_, max_speed, desired_min_space, vehicle_length)
    
    is_crashed = is_crashed | is_crashed_new

    """ Dynamic Friction Simulator """
    patch_locs, patch_sizes, friction_values, is_growing = surface_friction_dynamics(patch_locs=patch_locs, 
                                                                                     patch_sizes=patch_sizes,
                                                                                       friction_values=friction_values,
                                                                                         is_growings=is_growings,
                                                                                           road_length=ring_radius * 2 * np.pi)
    # get friction values for each vehicle
    friction_trues = get_friction(x, patch_locs, patch_sizes, friction_values)
    # make them noisy
    friction_measurements = friction_trues + np.random.normal(0, 0.2, num_vehicles)

    test_x = np.linspace(0, ring_radius * 2 * np.pi, 1000)
    test_friction = get_friction(test_x, patch_locs, patch_sizes, friction_values)
    # Create a colormap
    cmap = plt.get_cmap('coolwarm')
    norm = plt.Normalize(0, 0.8)
    test_X = np.cos(test_x/ring_radius)
    test_Y = np.sin(test_x/ring_radius)

    # Create line segments
    points = np.array([test_X, test_Y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # Create the LineCollection object
    lc = LineCollection(segments, cmap=cmap, norm=norm, linewidths=8, alpha=.5)
    lc.set_array(test_friction)  # Set the colors according to z
    
    # friction_trues = 1

    a = np.clip(driver_accelerations, -9.81*friction_trues, 9.81*friction_trues)

    x += v * dt
    v += a * dt

    t += dt
    data[i, :, 0] = x
    data[i, :, 1] = v

    
    # add the number of vehicles that have crashed to a value
    num_crashed = np.sum(is_crashed)

    # make status_color array to default blue
    status_color = np.full(num_vehicles, 'b')

    # make the color of the vehicles that have abs(dirver_accelerations) more than value a yellow to indicate skidding
    status_color = np.where(np.abs(driver_accelerations) > np.abs(a), 'y', status_color) 

    # make the status_color red for vehicles that have crashed
    status_color = np.where(is_crashed, 'r', status_color)

    if online_plot:
        ax.cla()

        ax.add_collection(lc)
        ax.scatter(np.cos(x/ring_radius), np.sin(x/ring_radius), c=status_color, cmap='viridis')
        ax.set_xlim(-2, 2)
        ax.set_ylim(-2, 2)
        ax.set_title("Number of Crashes: {}".format(num_crashed))
        # plt.axes().axis('equal')
        # axcb = fig.colorbar(lc)  # Add a colorbar
        # axcb.set_label('Surface friction coefficient')


        # ax.plot(test_x, test_friction, color='k', alpha=0.5)
        # ax.set_title("Friction")
        # ax.set_ylim(0, 1)


        plt.pause(0.001)

# build a set of colors using num_vehicles
colors = plt.cm.viridis(np.linspace(0, 1, num_vehicles))

# make two subplots
fig, axs = plt.subplots(2)

for i in range(num_vehicles):

    axs[0].plot(data[:, i, 0], color='k', alpha=0.1)
    axs[0].set_title("Position")

    axs[1].plot(data[:, i, 1], color='k', alpha=0.1)
    axs[1].set_title("Velocity")




Initial conditions set
MinDistances:  0.026894901983780528


  0%|          | 0/100000 [00:00<?, ?it/s]C:\Users\Hossein_Haeri\AppData\Local\Temp\ipykernel_10088\1644294118.py:194: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(np.cos(x/ring_radius), np.sin(x/ring_radius), c=status_color, cmap='viridis')
  2%|▏         | 2034/100000 [01:49<1:27:35, 18.64it/s]


KeyboardInterrupt: 

In [54]:
patch_locs, patch_sizes, friction_values, is_growings = surface_friction_dynamics(road_length=ring_radius * 2 * np.pi)


[56169.83658875 41471.45056349 35268.39809694 31972.50493803
 53177.09129792 37798.17129174 42657.34454916 46669.15577518
 36663.2131067  57268.72713784]
[ 3.08604682 34.23635659 14.12578792 49.82402009 22.31620435 14.27386902
 22.10146878  0.44333    30.86656915  6.70826364]
None
